# Production Lifecycle Iterations

Let's now explore the options for any iterations in the production lifecycle.

In [ ]:
%cd ../forml-solution-avazuctr

## Monitoring the Serving Performance

The foremost action is monitoring the serving performance by evaluating the predictions against the eventually obtained true outcomes:

In [2]:
... # Evaluate the forml-solution-avazuctr serving performance of predictions made after '2014-10-21 03:00:00'

0.38597474427877504


## Model Refreshing (Update)

As time goes, the model is going to start indicating drift. Updating it with the recent data should be part of the regular process. With ForML, this involves producing new _generation_ of the model:

In [3]:
... # Train a new generation of forml-solution-avazuctr 0.1 on data between '2014-10-21 03:00:00' and '2014-10-21 05:00:00' 

In [ ]:
! forml model list forml-solution-avazuctr 0.1

Wait for the _serving gateway_ to pick up the newer model (given its default `latest` strategy) and try a new request:

In [5]:
! curl -H 'Content-Type: application/json' -d '[{ \
    "hour": "2014-10-21 05:00:00", \
    "banner_pos": "0", \
    "site_id": "887a4754", "site_domain": "e3d9ca35", \
    "site_category": "50e219e0", \
    "app_id": "ecad2386", "app_domain": "7801e8d9", \
    "app_category": "07d7df22", \
    "device_id": "0e79d423", "device_ip": "9f423918", \
    "device_model": "fc10a0d3", \
    "device_type": "0", "device_conn_type": "0", \
    "C1": "1002", "C14": "22701", "C15": "320", "C16": "50", \
    "C17": "2624", "C18": "0", "C19": "35", "C20": "-1", "C21": "221" \
}]' http://127.0.0.1:8000/forml-solution-avazuctr

[{"c0":0.1343483272}]

The performance monitoring for the next period would then go like this:

In [ ]:
! forml model eval forml-solution-avazuctr \
    --lower '2014-10-21 05:00:00'

## New Model Release (Upgrade)

There can be a number of reasons why just refreshing the model might not bring the required improvements and a true gain would only be possible through a conceptually new version of the (logical) model (i.e. its code). This involves a new development iteration(s) and eventually a new release of the model.

We will demonstrate this process by an attempt to simplify the model by removing some of the not-so-useful columns.

Let's tap into our pipeline just after the `TargetEncoder` to be able to analyze that data:

In [7]:
from forml import project
from forml.pipeline import payload, wrap
from avazuctr import pipeline

with wrap.importer():
    from category_encoders import TargetEncoder

PROJECT = project.open(path=".", package="avazuctr")
trainset = PROJECT.components.source.bind(
    TargetEncoder(cols=pipeline.CATEGORICAL_COLUMNS)
).launcher.train()

Now we can simply calculate the pairwise feature correlations and filter anything above 0.9:

In [ ]:
import pandas
corr = trainset.features.corr()
corr[corr > 0.90].dropna(thresh=2).dropna(thresh=2, axis=1)

### Removing the Redundant Features

We see strong correlations between the following features:
* `device_type` and `C1`
* `site_domain` and `site_id`
* `C14` and `C17` and `C21`
* `C15` and `C16`

Let's update our [avazuctr/source.py](../forml-solution-avazuctr/avazuctr/source.py) and [avazuctr/pipeline.py](../forml-solution-avazuctr/avazuctr/pipeline.py) to keep only the first feature from each of the sets:

#### Updating source.py

1. Open the [avazuctr/source.py](../forml-solution-avazuctr/avazuctr/source.py) component.
2. Edit the `FEATURES` sequence to remove the `device_type`, `site_domain`, `C15`, `C17`, and `C21` features.
3. **SAVE THE [avazuctr/source.py](../forml-solution-avazuctr/avazuctr/source.py) FILE!**

In [9]:
! git add avazuctr/source.py

#### Updating pipeline.py
1. Open the [avazuctr/pipeline.py](../forml-solution-avazuctr/avazuctr/pipeline.py) component.
2. Edit the`CATEGORICAL_COLUMNS` sequence to remove the `device_type`, `site_domain`, `C15`, `C17`, and `C21` features.
3. **SAVE THE [avazuctr/pipeline.py](../forml-solution-avazuctr/avazuctr/pipeline.py) FILE!**

In [10]:
! git add avazuctr/pipeline.py

Now we can evaluate the impact of this change:

In [ ]:
! forml project eval

This even comes with a slightly improved loss!

### Releasing the New Model

Incrementing Project Version:

1. Open the [pyproject.toml](../forml-solution-avazuctr/pyproject.toml).
2. Edit the `version` setting it to `0.2`:
```toml
version = "0.2"
```
3. **SAVE THE [pyproject.toml](../forml-solution-avazuctr/pyproject.toml) FILE!**

In [12]:
! git add pyproject.toml

Commit and tag the code:

In [ ]:
! git commit -m 'Released 0.2'
! git tag 0.2

Kick off the packaging and model publishing:

In [ ]:
! forml project release

We should now see the new release in the registry:

In [ ]:
! forml model list forml-solution-avazuctr

Let's train first generation model of this new release:

In [16]:
... # Train first generation of forml-solution-avazuctr 0.2 on data before '2014-10-21 03:00:00'

In [ ]:
! forml model list forml-solution-avazuctr 0.2

In [18]:
! curl -H 'Content-Type: application/json' -d '[{ \
    "hour": "2014-10-21 03:00:00", \
    "banner_pos": "0", \
    "site_id": "887a4754", "site_domain": "e3d9ca35", \
    "site_category": "50e219e0", \
    "app_id": "ecad2386", "app_domain": "7801e8d9", \
    "app_category": "07d7df22", \
    "device_id": "0e79d423", "device_ip": "9f423918", \
    "device_model": "fc10a0d3", \
    "device_type": "0", "device_conn_type": "0", \
    "C1": "1002", "C14": "22701", "C15": "320", "C16": "50", \
    "C17": "2624", "C18": "0", "C19": "35", "C20": "-1", "C21": "221" \
}]' http://127.0.0.1:8000/forml-solution-avazuctr

[{"c0":0.2057745536}]

In [19]:
! forml model eval forml-solution-avazuctr \
    --lower '2014-10-21 03:00:00'

0.3866724370765236


## A/B Testing Multiple Models

Since we now have multiple model instances in our registry, we might want to change the selection strategy from the default _latest_ to for example A/B testing. It takes only slight tweaking of the application descriptor.

Let's change the strategy so that the models get selected according to the following plan (`target` is the weight this model should end up being selected with):

| Release | Generation | Target |
|---------|------------|--------|
| `0.1`   | `1`        | 3      |
| `0.1`   | `2`        | 5      |
| `0.2`   | `1`        | 2      |

Now, add the generic application descriptor code to the [application.py](../forml-solution-avazuctr/application.py):

1. Open the [application.py](../forml-solution-avazuctr/application.py) component.
2. Update it with the following code defining the `ABTest` selector strategy:
```python
from forml import application

selector = (
    application.ABTest.compare(
        project="forml-solution-avazuctr",
        release="0.1",
        generation=1,
        target=3,                                    # variant A 30%
    )
    .over(generation=2, target=5)                    # variant B 50%
    .against(release="0.2", generation=1, target=2)  # variant C 20%
)

application.setup(
    application.Generic("forml-solution-avazuctr", selector)
)
```
3. **SAVE THE [application.py](../forml-solution-avazuctr/application.py) FILE!**

In [20]:
! git add application.py

### Redeploying the Application

In [21]:
... # Redeploy the application descriptor application.py

> **!Restart the serving gateway!**

Make a few requests and watch the `x-forml-instance` header showing the model:

In [22]:
! curl -s -v -H 'Content-Type: application/json' -d '[{ \
    "hour": "2014-10-21 03:00:00", \
    "banner_pos": "0", \
    "site_id": "887a4754", "site_domain": "e3d9ca35", \
    "site_category": "50e219e0", \
    "app_id": "ecad2386", "app_domain": "7801e8d9", \
    "app_category": "07d7df22", \
    "device_id": "0e79d423", "device_ip": "9f423918", \
    "device_model": "fc10a0d3", \
    "device_type": "0", "device_conn_type": "0", \
    "C1": "1002", "C14": "22701", "C15": "320", "C16": "50", \
    "C17": "2624", "C18": "0", "C19": "35", "C20": "-1", "C21": "221" \
}]' http://127.0.0.1:8000/forml-solution-avazuctr \
    2> >(grep x-forml-instance)

< x-forml-instance: dispatch-registry-forml-solution-avazuctr-0.1-2
[{"c0":0.1364936214}]